### The complete code

In [5]:
print(len(node2idx))

26868


In [6]:
rel2idx

{'RegDeleteKey': 0,
 'Process Create': 1,
 'ReadFile': 2,
 'TCP Accept': 3,
 'RegOpenKey': 4,
 'SetDispositionInformationFile': 5,
 'TCP Send': 6,
 'CloseFile': 7,
 'CreateFile': 8,
 'RegSetInfoKey': 9,
 'SetBasicInformationFile': 10,
 'RegCreateKey': 11,
 'QueryBasicInformationFile': 12,
 'TCP Connect': 13,
 'WriteFile': 14,
 'CreateFileMapping': 15,
 'NO CORRESPONDING RELATION': 16,
 'Thread Create': 17,
 'TCP Receive': 18,
 'NO CORRESPONDING RELATION(EVENT OTHER)': 19,
 'SetRenameInformationFile': 20,
 'Load Image': 21,
 'NO CORRESPONDING RELATION(EVENT BIND)': 22}

In [12]:
# print(node2idx[:10])
count = 0
for item in node2idx:
    print(item)
    print()
    count += 1
    if count == 5:break

\REGISTRY\MACHINE\SOFTWARE\Microsoft\.NETFramework\v2.0.50727\NGenService\Roots\Microsoft.VisualStudio.Tools.Applications.Hosting,_Version=10.0.0.00000,_Culture=neutral,_PublicKeyToken=B03F5F7F11D50A3A\0

\REGISTRY\MACHINE\SOFTWARE\Microsoft\Windows_Search\CrawlScopeManager\Windows\SystemIndex\SearchRoots\0

\REGISTRY\MACHINE\SOFTWARE\Classes\IMECheckDefaultInputProfile.Japan.3

\REGISTRY\USER\S-1-5-21-231540947-922634896-4161786520-1004\Software\Microsoft\Windows\CurrentVersion\ContentDeliveryManager\Health\Placement-SubscribedContent-338388

\REGISTRY\MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\Component_Based_Servicing\Packages\Microsoft-Windows-HyperV-Guest-Package~31bf3856ad364e35~amd64~~10.0.16299.15\Owners



In [13]:
print(type(node2idx))

<class 'dict'>


In [14]:
print(type(all_events))

<class 'list'>


In [15]:
print(len(all_events))

18535675


In [17]:
# print(all_events[:5])
for event in all_events[:5]:
    print(event, end='\n')  # 默认情况，end='\n' 可省略，因为默认就是换行符

{"subj": {"uuid": "000658F8-A42A-4B44-9C9E-19CFADB72C37", "type": "Subject", "n_attrbiute": {"cmdline": "None", "type": "SUBJECT_THREAD", "pid": 6856}}, "relation": "EVENT_CHECK_FILE_ATTRIBUTES", "obj": {"uuid": "DB470BCE-EFFC-4BD8-8A06-B826C34FB2C1", "type": "FileObject", "n_attrbiute": {"filepath": "\\Device\\HarddiskVolume2\\Windows\\System32\\Windows.StateRepositoryPS.dll"}}, "timestamp": 1523461670758000000, "label": "benign"}

{"subj": {"uuid": "000658F8-A42A-4B44-9C9E-19CFADB72C37", "type": "Subject", "n_attrbiute": {"cmdline": "None", "type": "SUBJECT_THREAD", "pid": 6856}}, "relation": "EVENT_CHECK_FILE_ATTRIBUTES", "obj": {"uuid": "DB470BCE-EFFC-4BD8-8A06-B826C34FB2C1", "type": "FileObject", "n_attrbiute": {"filepath": "\\Device\\HarddiskVolume2\\Windows\\System32\\Windows.StateRepositoryPS.dll"}}, "timestamp": 1523461670758000000, "label": "benign"}

{"subj": {"uuid": "000658F8-A42A-4B44-9C9E-19CFADB72C37", "type": "Subject", "n_attrbiute": {"cmdline": "None", "type": "SUBJE

In [2]:
from tqdm import tqdm
import json
from glob import glob
import linecache

# 读取关系映射
rel2idx = {}
with open('/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/relation2id.txt', 'r') as f:
    next(f)  # 跳过第一行（总数）
    for line in tqdm(f):
        relation, idx = line.strip().split('\t')
        rel2idx[relation] = int(idx)

# 读取实体映射
node2idx = {}
with open('/workdir/home/bai/Euni_HO_modified/data/3_openKE/synthesize/entity2id.txt', 'r') as f:
    next(f)  # 跳过第一行（总数）
    for line in tqdm(f):
        entity, idx = line.strip().split('\t')
        node2idx[entity] = int(idx)

# 读取并排序所有事件
file_path = glob(f'../data_new/DARPA/source_data/3.4/*.json')
file_path = sorted(file_path)

all_events = []
for p in tqdm(file_path):
    all_events.extend(linecache.updatecache(p))

all_events = sorted(all_events)

# 使用映射转换事件数据
event_encode = []
for e in tqdm(all_events, desc="Encoding events"):
    e = json.loads(e)
    label = 0 if e["label"] == "benign" else 1
    src_entity = e["subj"]["uuid"]
    dst_entity = e["obj"]["uuid"]
    relation = e["relation"]

    if src_entity in node2idx and dst_entity in node2idx and relation in rel2idx:
        event_encode.append((node2idx[src_entity], node2idx[dst_entity], rel2idx[relation], label))
    else:
        pass  # 您可以在这里处理找不到映射的情况

# 保存转换后的数据
with open('../data_new/DARPA/before_embedding/3.10/event_encode.txt', 'w') as f:
    for item in tqdm(event_encode, desc="Saving encoded events"):
        f.write(f'{item[0]} {item[1]} {item[2]} {item[3]}\n')

# 计算 label = 1 的事件数量
label_1_count = sum(1 for event in event_encode if event[-1] == 1)

# 打印 label = 1 的事件数量
print("Number of events with label = 1:", label_1_count)

# 打印 label = 1 的所有事件
print("Events with label = 1:")
for event in event_encode:
    if event[-1] == 1:
        print(event)


23it [00:00, 134171.06it/s]
26868it [00:00, 493696.13it/s]
Encoding events: 100%|██████████| 18535675/18535675 [02:09<00:00, 142773.73it/s]
Saving encoded events: 0it [00:00, ?it/s]

Number of events with label = 1: 0
Events with label = 1:


In [4]:
print(event_encode)

[]


In [ ]:
from glob import glob
import linecache
import json
from tqdm import tqdm

file_path = glob(f'../data_new/DARPA/source_data/3.4/*.json')
file_path = sorted(file_path)

all_events = []
for p in tqdm(file_path):
    all_events.extend(linecache.updatecache(p))

all_events = sorted(all_events)

# subj 和 obj 的 uuids
nodes = set()
rels = set()

for e in tqdm(all_events, desc="Processing events"):
    e = json.loads(e)

    nodes.add(e["subj"]["uuid"])
    nodes.add(e["obj"]["uuid"])
    rels.add(e["relation"])

node2idx = {uuid: idx for idx, uuid in enumerate(tqdm(nodes, desc="Mapping nodes"))}
rel2idx = {rel: idx for idx, rel in enumerate(tqdm(rels, desc="Mapping relations"))}


event_encode = []
for e in tqdm(all_events, desc="Encoding events"):
    e = json.loads(e)
    label = 0 if e["label"] == "benign" else 1
    event_encode.append((node2idx[e["subj"]["uuid"]], node2idx[e["obj"]["uuid"]], rel2idx[e["relation"]], label))

print(event_encode)
print(node2idx)

with open('../data_new/DARPA/before_embedding/3.10/node2idx.txt', 'w') as f:
    for key, value in tqdm(node2idx.items()):
        f.write(f'{key} {value}\n')

with open('../data_new/DARPA/before_embedding/3.10/rel2idx.txt', 'w') as f:
    for key, value in tqdm(rel2idx.items()):
        f.write(f'{key} {value}\n')

with open('../data_new/DARPA/before_embedding/3.10/event_encode.txt', 'w') as f:
    for item in tqdm(event_encode):
        f.write(f'{item[0]} {item[1]} {item[2]} {item[3]}\n')

# 计算 label = 1 的事件数量
label_1_count = sum(1 for event in event_encode if event[-1] == 1)

# 打印 label = 1 的事件数量
print("Number of events with label = 1:", label_1_count)

# 打印 label = 1 的所有事件
print("Events with label = 1:")
for event in event_encode:
    if event[-1] == 1:
        print(event)

### This is the code for change the data format from original json file to (src, dest, rel, label)

In [1]:
from glob import glob
import linecache
import json
from tqdm import tqdm

#### get the input file

In [3]:
file_path = glob(f'../data_new/DARPA/source_data/3.4/*.json')
file_path = sorted(file_path)

all_events = []
for p in tqdm(file_path):
    all_events.extend(linecache.updatecache(p))

all_events = sorted(all_events)

100%|██████████| 4/4 [00:12<00:00,  3.14s/it]


#### get the node, relation mapping

In [4]:
# subj 和 obj 的 uuids
nodes = set()
rels = set()

for e in tqdm(all_events, desc="Processing events"):
    e = json.loads(e)

    nodes.add(e["subj"]["uuid"])
    nodes.add(e["obj"]["uuid"])
    rels.add(e["relation"])

node2idx = {uuid: idx for idx, uuid in enumerate(tqdm(nodes, desc="Mapping nodes"))}
rel2idx = {rel: idx for idx, rel in enumerate(tqdm(rels, desc="Mapping relations"))}

Mapping relations: 100%|██████████| 25/25 [00:00<00:00, 190650.18it/s]


#### encode the output file in the format we want

In [14]:
event_encode = []
for e in tqdm(all_events, desc="Encoding events"):
    e = json.loads(e)
    label = 0 if e["label"] == "benign" else 1
    event_encode.append((node2idx[e["subj"]["uuid"]], node2idx[e["obj"]["uuid"]], rel2idx[e["relation"]], label))

print(event_encode)
print(node2idx)

Encoding events: 100%|██████████| 18535675/18535675 [02:29<00:00, 124160.75it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### output the txt file

In [15]:
with open('../data_new/DARPA/before_embedding/3.10/node2idx.txt', 'w') as f:
    for key, value in tqdm(node2idx.items()):
        f.write(f'{key} {value}\n')

with open('../data_new/DARPA/before_embedding/3.10/rel2idx.txt', 'w') as f:
    for key, value in tqdm(rel2idx.items()):
        f.write(f'{key} {value}\n')

with open('../data_new/DARPA/before_embedding/3.10/event_encode.txt', 'w') as f:
    for item in tqdm(event_encode):
        f.write(f'{item[0]} {item[1]} {item[2]} {item[3]}\n')


#### show the statistics of the labels

In [9]:
# 计算 label = 1 的事件数量
label_1_count = sum(1 for event in event_encode if event[-1] == 1)

# 打印 label = 1 的事件数量
print("Number of events with label = 1:", label_1_count)

# 打印 label = 1 的所有事件
print("Events with label = 1:")
for event in event_encode:
    if event[-1] == 1:
        print(event)

Number of events with label = 1: 861
Events with label = 1:
(130792, 10, 95612, 1)
(130792, 10, 95612, 1)
(130792, 10, 95612, 1)
(130792, 21, 79663, 1)
(130792, 20, 79663, 1)
(111855, 10, 87952, 1)
(111855, 10, 87952, 1)
(111855, 10, 87952, 1)
(111855, 10, 14562, 1)
(111855, 10, 14562, 1)
(111855, 10, 14562, 1)
(111855, 10, 52751, 1)
(111855, 10, 52751, 1)
(111855, 10, 52751, 1)
(111855, 10, 45201, 1)
(111855, 10, 45201, 1)
(111855, 10, 45201, 1)
(111855, 10, 64339, 1)
(111855, 10, 64339, 1)
(111855, 10, 64339, 1)
(111855, 20, 87952, 1)
(111855, 20, 87952, 1)
(111855, 20, 87952, 1)
(111855, 20, 14562, 1)
(111855, 20, 14562, 1)
(111855, 20, 14562, 1)
(111855, 20, 52751, 1)
(111855, 20, 52751, 1)
(111855, 20, 52751, 1)
(111855, 20, 45201, 1)
(111855, 20, 45201, 1)
(111855, 20, 45201, 1)
(111855, 20, 64339, 1)
(111855, 20, 64339, 1)
(111855, 20, 64339, 1)
(135123, 10, 52751, 1)
(135123, 10, 52751, 1)
(135123, 10, 52751, 1)
(135123, 20, 52751, 1)
(135123, 20, 52751, 1)
(135123, 20, 52751, 

(107562, 10, 64339, 1)
(107562, 20, 14562, 1)
(107562, 20, 14562, 1)
(107562, 20, 14562, 1)
(107562, 20, 52751, 1)
(107562, 20, 52751, 1)
(107562, 20, 52751, 1)
(107562, 20, 45201, 1)
(107562, 20, 45201, 1)
(107562, 20, 45201, 1)
(107562, 20, 64339, 1)
(107562, 20, 64339, 1)
(107562, 20, 64339, 1)
(5686, 10, 87952, 1)
(5686, 10, 87952, 1)
(5686, 10, 87952, 1)
(5686, 10, 14562, 1)
(5686, 10, 14562, 1)
(5686, 10, 14562, 1)
(5686, 10, 52751, 1)
(5686, 10, 52751, 1)
(5686, 10, 52751, 1)
(5686, 10, 45201, 1)
(5686, 10, 45201, 1)
(5686, 10, 45201, 1)
(5686, 10, 64339, 1)
(5686, 10, 64339, 1)
(5686, 10, 64339, 1)
(5686, 20, 87952, 1)
(5686, 20, 87952, 1)
(5686, 20, 87952, 1)
(5686, 20, 14562, 1)
(5686, 20, 14562, 1)
(5686, 20, 14562, 1)
(5686, 20, 52751, 1)
(5686, 20, 52751, 1)
(5686, 20, 52751, 1)
(5686, 20, 45201, 1)
(5686, 20, 45201, 1)
(5686, 20, 45201, 1)
(5686, 20, 64339, 1)
(5686, 20, 64339, 1)
(5686, 20, 64339, 1)
(109686, 10, 87952, 1)
(109686, 10, 87952, 1)
(109686, 10, 87952, 1)
(1

In [13]:
label_count = len(event_encode)

print(f"Benign : Malicious = {label_count - label_1_count} : {label_1_count}")
print(f"Ratio: {label_1_count/label_count}")

Benign : Malicious = 18534814 : 861
Ratio: 4.6450965503009734e-05
